In [1]:
import ta
import time
import math
import pprint
import asyncio
import datetime
import itertools
import pandas as pd
import SecretConfig
import ccxt.pro as ccxt
from IPython.display import clear_output

In [12]:
# §§§§§§§§§§

await exchange.close()

# §§§§§§§§§§

In [2]:
exchange = ccxt.bitget({
    "apiKey":SecretConfig.APIKey,
    "secret":SecretConfig.SecretKey,
    'password': SecretConfig.Passphrase,
    "options": {'defaultType': 'swap', 'adjustForTimeDifference' : True},
    "timeout": 60000,
    "enableRateLimit": True,
})

In [3]:
# LIVE PRICE ET MARKETS

async def live_prices() :

    global last_20
    global tickers
    global swap_markets
  
    tickers = await exchange.fetch_tickers()

    await asyncio.sleep(0.005)

    timestamp = list(tickers.values())[0]['timestamp']
    index = [datetime.datetime.fromtimestamp(timestamp/1000).replace(microsecond=0)]

    data = {}

    for symbol, ticker in tickers.items():

        data[symbol +  " last"] = [ticker['last']]
        data[symbol +  " ask"] = [ticker['ask']]
        data[symbol +  " bid"] = [ticker['bid']]

    last_prices = pd.DataFrame(data, index=index)
    last_20 = pd.concat([last_20, last_prices])
    last_20 = last_20.tail(wind_EMA2 + wind_Var_EMA2 + wind_Div_EMA1_2 + wind_EMA2)

    markets = await exchange.load_markets()

    await asyncio.sleep(0.005)

    swap_markets = {symbol: market for symbol, market in markets.items() if market['swap']}

In [4]:
# CALCUL DES INDICATEURS TECHNIQUES

async def indicators_calculation() :
    
    global data1
    global data2
    global data3 
    global data4 
    global data5
    global data6
    global data7
    global data8

    global top_10_values

    global fulldata
       
    fulldata_row = pd.DataFrame()

    index = [last_20.index[-1]] 

    data_row_step1 = pd.DataFrame({symbol + suffix:
                                   [last_20[symbol + " last"].iloc[-1]] if suffix == " last" else 
                                   [last_20[symbol + " ask"].iloc[-1]] if suffix == " ask" else 
                                   [last_20[symbol + " bid"].iloc[-1]] if suffix == " bid" else 
                                   [((last_20[symbol + " last"].iloc[-1] - last_20[symbol + " last"].iloc[- wind_Var_Price]) /
                                     last_20[symbol + " last"].iloc[- wind_Var_Price]) * 100]  if suffix == " Var Last Price" and len(last_20) > wind_Var_Price else
                                     [ta.trend.ema_indicator(last_20[symbol + ' last'], window= wind_EMA1).iloc[-1]] if suffix == ' EMA_1' and len(last_20) > wind_EMA1 else
                                     [ta.trend.ema_indicator(last_20[symbol + ' last'], window= wind_EMA2).iloc[-1]] if suffix == ' EMA_2' and len(last_20) > wind_EMA2 else 
                                     [None] for symbol, ticker in tickers.items() for suffix in 
                                     [" last", " ask", " bid", ' Var Last Price', ' EMA_1', ' EMA_2']}, index=index)

                                
    data_row_step1 = data_row_step1.apply(pd.to_numeric)
    data1 = pd.concat([data1, data_row_step1])
    data1 = data1.tail(wind_Var_EMA1 + wind_Var_EMA2 + 1)
    # await asyncio.sleep(0.01)

   

    data_row_step2 = pd.DataFrame({symbol + suffix:
                                   (data1[symbol + " Var Last Price"].iloc[-1])**2 if suffix == ' Var Carre Last Price' and len(last_20) > wind_Var_Price else
                                   ((data1[symbol + ' EMA_1'].iloc[-1] - data1[symbol + ' EMA_1'].iloc[- wind_Var_EMA1]) /
                                    data1[symbol + ' EMA_1'].iloc[- wind_Var_EMA1]) * 100  if suffix == ' Var EMA_1' and len(last_20) > wind_EMA1 + wind_Var_EMA1  else
                                    ((data1[symbol + ' EMA_2'].iloc[-1] - data1[symbol + ' EMA_2'].iloc[- wind_Var_EMA2]) /
                                     data1[symbol + ' EMA_2'].iloc[- wind_Var_EMA2]) * 100  if suffix == ' Var EMA_2' and len(last_20) > wind_EMA2 + wind_Var_EMA2 else
                                     (data1[symbol + " last"].iloc[- 1] - data1[symbol + ' EMA_1'].iloc[-1]) ** 2 if suffix == ' Diff Carre Price_EMA_1' and len(last_20) > wind_EMA1 else
                                     (data1[symbol + ' EMA_1'].iloc[- 1] - data1[symbol + ' EMA_2'].iloc[-1]) ** 2 if suffix == ' Diff Carre EMA_1_2' and len(last_20) > wind_EMA2 else
                                     [None] for symbol, ticker in tickers.items() for suffix in 
                                     [' Var Carre Last Price', ' Var EMA_1', ' Var EMA_2', ' Diff Carre Price_EMA_1', ' Diff Carre EMA_1_2']}, index=index)
    
    data_row_step2 = data_row_step2.apply(pd.to_numeric)
    data2 = pd.concat([data2, data_row_step2])
    data2 = data2.tail(wind_Div_Price_EMA1 + wind_Div_EMA1_2 + 1)
    # await asyncio.sleep(0.01)
    
    data_row_step3 = pd.DataFrame({symbol+ suffix:
                                   ((data2[symbol + ' Diff Carre Price_EMA_1'].iloc[-1] - data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1]) /
                                    data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1]) * 100  if suffix == ' Divergence Price_EMA_1' and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 and data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1] != 0 else
                                    ((data2[symbol + ' Diff Carre EMA_1_2'].iloc[-1] - data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2]) /
                                    data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2]) * 100  if suffix == ' Divergence EMA_1_2' and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 and data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2] != 0 else
                                    [None] for symbol, ticker in tickers.items() for suffix in 
                                    [' Divergence Price_EMA_1', ' Divergence EMA_1_2']}, index=index)
    
    data_row_step3 = data_row_step3.apply(pd.to_numeric)
    data3 = pd.concat([data3, data_row_step3])
    data3 = data3.tail(wind_Div_Moy_Price_EMA1 + wind_Div_Moy_EMA1_2 + 1)
    # await asyncio.sleep(0.01)

    data_row_step4 = pd.DataFrame({symbol + suffix:
                                   ta.trend.ema_indicator(data3[symbol + ' Divergence Price_EMA_1'], window= wind_Div_Moy_Price_EMA1).iloc[-1] if suffix == ' Divergence Moyen Price_EMA_1' and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 else 
                                   ta.trend.ema_indicator(data3[symbol + ' Divergence EMA_1_2'], window= wind_Div_Moy_EMA1_2).iloc[-1] if suffix == ' Divergence Moyen EMA_1_2' and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2  else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Divergence Moyen Price_EMA_1', ' Divergence Moyen EMA_1_2']}, index=index)
    
    data_row_step4 = data_row_step4.apply(pd.to_numeric)
    data4 = pd.concat([data4, data_row_step4])
    data4 = data4.tail(wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2)
    # await asyncio.sleep(0.01)

    data_row_step5 = pd.DataFrame({symbol + suffix:
                                   data4[symbol + ' Divergence Moyen Price_EMA_1'].iloc[-1] > 0 and data4[symbol + ' Divergence Moyen EMA_1_2'].iloc[-1]  > 0 if suffix == ' Condition'
                                   and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 else
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Condition']}, index=index)
    
    # data_row_step5 = data_row_step5.apply(pd.to_numeric)
    data5 = pd.concat([data5, data_row_step5])
    # data5 = data5.apply(pd.to_numeric)
    data5 = data5.tail(5)
    # clear_output(wait=True)
    # display(data5.tail(2))
    # await asyncio.sleep(0.01)


    top_div = {}
    top_10_values = {}

    for symbol, ticker in tickers.items():
        if len(last_20) > wind_EMA1 + wind_Div_Price_EMA1:
            if data3[symbol + ' Divergence Price_EMA_1'].iloc[-1] > div_min_top:
                top_div[symbol] = (data3[symbol + ' Divergence Price_EMA_1'].iloc[-1], ticker['info']['symbol'])

    if top_div:
        top_10_values = dict(sorted(top_div.items(), key=lambda item: item[1][0], reverse=True)[:10])

    # display(f'top is : {top_10_values}')
  
    data_row_step6 = pd.DataFrame({symbol + suffix:
                                   1 if suffix == ' Top_10' and top_10_values is not None and symbol in top_10_values.keys() and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Top_10']}, index=index)
    
    data_row_step6 = data_row_step6.apply(pd.to_numeric)
    data6 = pd.concat([data6, data_row_step6])
    data6 = data6.tail(occur_wind)
    # clear_output(wait=True)
    '''if len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 :
        display(data6.tail(2))'''
    # await asyncio.sleep(0.01)

    data_row_step7 = pd.DataFrame({symbol + suffix: 
                                   data6[symbol + ' Top_10'].sum() if suffix == ' Occur' and len(fulldata) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + occur_wind + 1 else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Occur']}, index=index)
    data_row_step7 = data_row_step7.apply(pd.to_numeric)
    data7 = pd.concat([data7, data_row_step7])
    data7 = data7.tail(2)
    '''if len(fulldata) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + occur_wind :
        display(data7)'''
    # await asyncio.sleep(0.01)

    data_row_step8 = pd.DataFrame({symbol+ suffix:
                                   "open_long" if suffix == ' Tradable' and data7[symbol +' Occur'].iloc[-1] is not None and data7[symbol +' Occur'].iloc[-1] >
                                     min_occur and data5[symbol + ' Condition'].iloc[-1] and
                                   data1[symbol + ' Var Last Price'].iloc[-1] > 0 and data1[symbol + " last"].iloc[-1] >
                                     data1[symbol + ' EMA_1'].iloc[-1] and len(fulldata) >
                                       wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind  else
                                   "open_short" if suffix == ' Tradable' and data7[symbol +' Occur'].iloc[-1] is not None and data7[symbol +' Occur'].iloc[-1] >
                                     min_occur and data5[symbol + ' Condition'].iloc[-1] and
                                   data1[symbol + ' Var Last Price'].iloc[-1] < 0 and data1[symbol + " last"].iloc[-1] <
                                     data1[symbol + ' EMA_1'].iloc[-1] and len(fulldata) > wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Tradable']}, index=index) 
    data8 = pd.concat([data8, data_row_step8])
    data8 = data8.tail(2)
    '''if len(fulldata) > wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind :
        display(data8)'''
    # await asyncio.sleep(0.01)


    fulldata_row = pd.concat([data_row_step1, data_row_step2, data_row_step3, data_row_step4, data_row_step5, data_row_step6, data_row_step7, data_row_step8], axis=1)
    fulldata_row = fulldata_row.apply(pd.to_numeric, errors='ignore')
    fulldata_row =fulldata_row.sort_index(axis=1)
    fulldata = pd.concat([fulldata, fulldata_row])
    fulldata = fulldata.tail(wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind + wind_EMA2)
    # clear_output(wait=True)
    # display(data_row_step8)
    # await asyncio.sleep(0.01)

In [5]:
# MISE A JOUR DE LA BALANCE

async def refresh_balance() :

    global balance
    global free_balance
    global alertnbr
    global alert_df

    try :

        balance_data = await exchange.fetch_balance()

        await asyncio.sleep(0.005)

        balance = float(balance_data['info'][0]['available'])
        free_balance = float(balance_data['free']['USDT'])
    
    except Exception as e:
        
        alert = {'alert': str(e)}
        alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} balance'])
        alert_df = pd.concat([alert_df, alert_data])
        alertnbr += 1

    if balance < 185 :

        display("Balance is less than 190, stopping the program.")
        exit()

    # display(f'Balance is {balance}')
    # await asyncio.sleep(0.01)

In [6]:
# CHECK EXISTING Order & SL V2

async def config_strikerV2() :

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop
    global alertnbr
    global alert_df

    if not df_open_trade_info.empty:

        for symbol in df_open_trade_info.index :

            response_ord = []

            params = {"stop" : True}
            response_ord.extend(await exchange.fetch_open_orders(symbol, params=params))
            await asyncio.sleep(0.01)
            response_ord.extend(await exchange.fetch_open_orders(symbol))
            await asyncio.sleep(0.01)
            response_pos = await exchange.fetch_positions([symbol])
            await asyncio.sleep(0.01)

            if len(response_pos) == 0 :

                if len(response_ord) == 2 and (('info' in resp[0] and 'side' in resp[0]['info'] and 'info' in resp[1] and 'side' in resp[1]['info'] 
                                            and resp[0]['info']['side'] != resp[1]['info']['side']) for resp in response_ord) :
                     
                     if df_open_trade_info.loc[symbol, 'status'] == "starting" :
                         
                         df_open_trade_info.loc[symbol, 'status'] = "waiting"
                
                if all(not ord for ord in response_ord) or len(response_ord) == 0:

                    if df_open_trade_info.loc[symbol, 'status'] != "first_closing" :
                    
                        df_open_trade_info.loc[symbol, 'status'] = "first_closing" 

                    elif df_open_trade_info.loc[symbol, 'status'] == "first_closing" :
                    
                        df_open_trade_info.loc[symbol, 'status'] = "close"
                        
                elif len(response_ord) == 1 and any(('info' in resp and 'side' in resp['info'] and 
                                               ('open_long' in resp['info']['side'] or 'open_short' in resp['info']['side'])) for resp in response_ord) :
                    
                    df_open_trade_info.loc[symbol, 'status'] = "set_new_sl"

                elif len(response_ord) == 1 and any(('info' in resp and 'side' in resp['info'] and 
                                               ('close_long' in resp['info']['side'] or 'close_short' in resp['info']['side'])) for resp in response_ord) :
                    
                    if df_open_trade_info.loc[symbol, 'status'] != "first_cancelling" :

                        check_response_pos = await exchange.fetch_positions([symbol])

                        await asyncio.sleep(0.01)

                        if len(check_response_pos) == 0 :
                            
                            df_open_trade_info.loc[symbol, 'status'] = "first_cancelling"

                            params = {'stop' : True, "planType" : "normal_plan"}

                            await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol, params= params)



                    elif df_open_trade_info.loc[symbol, 'status'] == "first_cancelling" :
                    
                        check_response_pos = await exchange.fetch_positions([symbol])

                        await asyncio.sleep(0.01)

                        if len(check_response_pos) == 0 :

                            try :
                                
                                params = {'stop' : True, "planType" : "normal_plan"}

                                await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol, params= params)

                                await asyncio.sleep(0.005)

                                df_open_trade_info.loc[symbol, 'status'] = "cancel"

                            except Exception as e :
                            
                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} conf 21'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                                display(f'trade {symbol} NOT IN EXCHANGE')

                                try :
                            
                                    await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol)

                                    await asyncio.sleep(0.005)

                                    df_open_trade_info.loc[symbol, 'status'] = "cancel"

                                except Exception as e: 
                            
                                    alert = {'alert': f'trade {symbol} NOT IN EXCHANGE {e}'}
                                    alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} conf 22'])
                                    alert_df = pd.concat([alert_df, alert_data])
                                    alertnbr += 1

                                    df_open_trade_info.loc[symbol, 'status'] = "CONF WARNING"

            elif len(response_pos) == 1 :

                if any('entryPrice'in pos for pos in response_pos) :
                
                    if response_ord and any(('info' in resp and 'side' in resp['info'] and 
                                             (df_open_trade_info.loc[symbol, 'sl_side'] == resp['info']['side'])) for resp in response_ord):
                         
                         df_open_trade_info.loc[symbol, 'status'] = "open"

                    else :

                        df_open_trade_info.loc[symbol, 'status'] = "set_new_sl"

                else : 

                    df_open_trade_info.loc[symbol, 'status'] = "CONF WARNING"

            await asyncio.sleep(0.005)

In [7]:
# CHECK TIMING Order to Position

async def time_striker() :

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop
    global alertnbr
    global alert_df
    global time_to_fill

    if not df_open_trade_info.empty:
    
        for symbol in df_open_trade_info.index :

            if df_open_trade_info["status"].loc[symbol] == "waiting" :

                response = []

                params = {"stop" : True}
                response.extend(await exchange.fetch_open_orders(symbol, params=params))
                await asyncio.sleep(0.005)
                response.extend(await exchange.fetch_open_orders(symbol))
                await asyncio.sleep(0.01)

                if len(response) == 2 :

                    # pprint.pprint(f'Trying Resp bloc time')

                    for resp in response :

                        if (resp['id'] and resp['info']['side'] == 'open_long' and  (datetime.timedelta(seconds=round((fulldata.index[-1] - datetime.datetime.fromtimestamp(
                            resp['timestamp']/1000)).total_seconds()))).total_seconds() > time_to_fill
                            ) or (resp['id'] and resp['info']['side'] == 'open_short' and (datetime.timedelta(seconds=round((fulldata.index[-1] - datetime.datetime.fromtimestamp(
                                resp['timestamp']/1000)).total_seconds()))).total_seconds() > time_to_fill) :

                                if df_open_trade_info["sl_id"].loc[symbol] and df_open_trade_info["id"].loc[symbol] :
                            
                                    try :
                                
                                        params = {'stop' : True, "planType" : "normal_plan"}

                                        await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol, params= params)

                                        display(f'trade SL {symbol} close for time')

                                        try :

                                            await exchange.cancel_order(df_open_trade_info["id"].loc[symbol], symbol, params= params)

                                            display(f'trade {symbol} close for time')

                                            df_open_trade_info.loc[symbol, "status"]= "cancel"

                                        except Exception as e:
                                     
                                            alert = {'alert': str(e)}
                                            alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} time 11'])
                                            alert_df = pd.concat([alert_df, alert_data])
                                            alertnbr += 1

                                            try :

                                                await exchange.cancel_order(df_open_trade_info["id"].loc[symbol], symbol)

                                                display(f'trade SL {symbol} close for time cause Exception')

                                                df_open_trade_info.loc[symbol, "status"]= "cancel"

                                            except Exception as e:
                                     
                                                alert = {'alert': str(e)}
                                                alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} time 12'])
                                                alert_df = pd.concat([alert_df, alert_data])
                                                alertnbr += 1

                                                df_open_trade_info.loc[symbol, "status"] = "set_new_sl"

                                    except Exception as e:
                                     
                                        alert = {'alert': str(e)}
                                        alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} time 2'])
                                        alert_df = pd.concat([alert_df, alert_data])
                                        alertnbr += 1

                                        try :

                                            await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol)

                                            display(f'trade SL {symbol} close for time cause Exception')

                                            try :

                                                await exchange.cancel_order(df_open_trade_info["id"].loc[symbol], symbol, params= params)

                                                display(f'trade {symbol} close for time')

                                                df_open_trade_info.loc[symbol, "status"]= "cancel"

                                            except Exception as e:
                                     
                                                alert = {'alert': str(e)}
                                                alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} time 21'])
                                                alert_df = pd.concat([alert_df, alert_data])
                                                alertnbr += 1

                                                try :

                                                    await exchange.cancel_order(df_open_trade_info["id"].loc[symbol], symbol)

                                                    display(f'trade SL {symbol} close for time cause Exception')

                                                    df_open_trade_info.loc[symbol, "status"]= "cancel"

                                                except Exception as e:
                                     
                                                    alert = {'alert': str(e)}
                                                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} time 22'])
                                                    alert_df = pd.concat([alert_df, alert_data])
                                                    alertnbr += 1

                                                    df_open_trade_info.loc[symbol, "status"]= "set_new_sl"

                                        except Exception as e:
                                                
                                            alert = {'alert': str(e)}
                                            alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} time 3'])
                                            alert_df = pd.concat([alert_df, alert_data])
                                            alertnbr += 1
                                            df_open_trade_info["status"].loc[symbol] = "TIME WARNING"

                                else :

                                    df_open_trade_info.loc[symbol, "status"]= "set_new_sl"
                                
                    await asyncio.sleep(0.005)

In [8]:
# NETTOYAGE df_open_trade_info

async def cleaner() :

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop

    if not df_open_trade_info.empty:

        for symbol in df_open_trade_info.index :

            if (df_open_trade_info.loc[symbol, 'status'] == "close" or df_open_trade_info.loc[symbol, 'status'] == "cancel" 
                or df_open_trade_info.loc[symbol, 'status'] == "bad-config" or df_open_trade_info.loc[symbol, 'status'] == "SL WARNING CLOSING"):
                
                df_closed_trade_info = pd.concat([df_closed_trade_info, df_open_trade_info.loc[[symbol]]])
                
                df_open_trade_info = df_open_trade_info.drop(symbol)

                nbr_open_trade -= 1
            
                await asyncio.sleep(0.005)

In [10]:
# MISE A JOUR SL 

async def sl_loader():

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop
    global alertnbr
    global alert_df

    if not df_open_trade_info.empty:

        for symbol in df_open_trade_info.index:

            list_change_sl = []

            if df_open_trade_info.loc[symbol, 'status'] != "waiting" and (df_open_trade_info.loc[symbol, 'sl_side'] == "close_long" and float(df_open_trade_info.loc[symbol, 'sl_triggerPrice']) < float(
                    exchange.price_to_precision(symbol, fulldata[df_open_trade_info.loc[symbol, 'sym'] + ' bid'].iloc[-1] * (1 - trail_stop) - (swap_markets[df_open_trade_info.loc[symbol, 'sym']]['precision']['price']*10))
                ) or df_open_trade_info.loc[symbol, 'sl_side'] == "close_short" and float(df_open_trade_info.loc[symbol, 'sl_triggerPrice']) > float(
                    exchange.price_to_precision(symbol, fulldata[df_open_trade_info.loc[symbol, 'sym'] + ' ask'].iloc[-1] * (1 + trail_stop) + (swap_markets[df_open_trade_info.loc[symbol, 'sym']]['precision']['price']*10))
                ) or df_open_trade_info.loc[symbol, 'status'] == "set_new_sl" or df_open_trade_info.loc[symbol, 'status'] == "warning_new_sl") :
                
                list_change_sl = {
                    'symbol': symbol,
                    "type": 'limit',
                    'side': df_open_trade_info['sl_side'].loc[symbol],
                    'amount': df_open_trade_info['amount'].loc[symbol],
                    "price": exchange.price_to_precision(symbol, fulldata[df_open_trade_info.loc[symbol, 'sym'] + ' bid'].iloc[-1] * (1 - trail_stop) - (
                            swap_markets[df_open_trade_info.loc[symbol, 'sym']]['precision']['price'] * 20)) if df_open_trade_info['sl_side'].loc[symbol] == "close_long" else exchange.price_to_precision(
                        symbol, fulldata[df_open_trade_info.loc[symbol, 'sym'] + ' ask'].iloc[-1] * (1 + trail_stop) + (swap_markets[df_open_trade_info.loc[symbol, 'sym']]['precision']['price'] * 20)),
                    'params': {
                        "marginCoin": "USDT",
                        'side': df_open_trade_info['sl_side'].loc[symbol],
                        "triggerPrice": exchange.price_to_precision(symbol, fulldata[df_open_trade_info.loc[symbol, 'sym'] + ' bid'].iloc[-1] * (1 - trail_stop) - (
                            swap_markets[df_open_trade_info.loc[symbol, 'sym']]['precision']['price']*10)) if df_open_trade_info['sl_side'].loc[symbol] == "close_long" else exchange.price_to_precision(
                            symbol, fulldata[df_open_trade_info.loc[symbol, 'sym'] + ' ask'].iloc[-1] * (1 + trail_stop) + (swap_markets[df_open_trade_info.loc[symbol, 'sym']]['precision']['price']*10)),
                        "triggerType":"market_price",
                        "reduceOnly" : True,
                        "leverage": "1",
                        "planType" : "normal_plan",
                        'clientOid': f'{df_open_trade_info["clientOid"].loc[symbol]}_slV_{df_open_trade_info["sl_version"].loc[symbol] + 1}'}}
                
                await asyncio.sleep(0.005)

                try :
                        
                    new_sl = await exchange.create_order(symbol, list_change_sl["type"], list_change_sl['side'],
                                                              list_change_sl['amount'], list_change_sl['price'], list_change_sl['params'])

                    await asyncio.sleep(0.01)

                    df_open_trade_info.loc[symbol, 'sl_triggerPrice'] = float(list_change_sl['params']['triggerPrice'])

                    df_open_trade_info.loc[symbol, 'sl_Execut_Price'] = float(list_change_sl["price"])

                    df_open_trade_info.loc[symbol, 'sl_clientOid'] = list_change_sl['params']['clientOid']
                
                    df_open_trade_info.loc[symbol, 'sl_version'] += 1

                    if new_sl["id"] :

                        df_open_trade_info.loc[symbol, "new_sl_id"] = new_sl["id"]

                        params = {'stop' : True, "planType" : "normal_plan"}

                        respord = await exchange.fetch_open_orders(symbol, params= params)

                        if any('info' in resp and 'side' in resp['info'] and ("close_long" in resp['info']['side'] or "close_short" in resp['info']['side']) for resp in respord) :

                            try :
                                
                                params = {'stop' : True, "planType" : "normal_plan"}

                                await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol, params= params)

                                await asyncio.sleep(0.005)

                                df_open_trade_info.loc[symbol, 'sl_id'] = df_open_trade_info.loc[symbol, 'new_sl_id']

                            except Exception as e :
                            
                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} sl1'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                                df_open_trade_info.loc[symbol, 'status'] = "SL WARNING"

                                display(f'trade SL {symbol} NOT IN EXCHANGE 1')

                                try :
                            
                                    await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol)

                                    await asyncio.sleep(0.005)

                                    df_open_trade_info.loc[symbol, 'sl_id'] = df_open_trade_info.loc[symbol, 'new_sl_id']

                                except Exception as e: 
                            
                                    alert = {'alert': f'trade {symbol} NOT IN EXCHANGE {e}'}
                                    alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} sl2'])
                                    alert_df = pd.concat([alert_df, alert_data])
                                    alertnbr += 1

                                    df_open_trade_info.loc[symbol, 'status'] = "SL WARNING"

                                    display(f'trade SL {symbol} NOT IN EXCHANGE 1.1')

                                    try : 
                                        params = {'stop' : True, "planType" : "normal_plan"}
                                        await exchange.cancel_order(df_open_trade_info["id"].loc[symbol], symbol, params= params)
                                        await asyncio.sleep(0.005)
                                        check_pos = await exchange.fetch_positions([symbol])
                                        if len(check_pos) == 0 :
                                            df_open_trade_info.loc[symbol, 'status'] = "SL WARNING CLOSING"
                                        else : 
                                            pprint.pprint(f'WARNING CLOSE {symbol} MANUALY !!!')

                                            df_open_trade_info.loc[symbol, 'sl_id'] = df_open_trade_info.loc[symbol, 'new_sl_id']
                                            # exit()

                                    except Exception :

                                        try : 

                                            await exchange.cancel_order(df_open_trade_info["id"].loc[symbol], symbol)
                                            await asyncio.sleep(0.005)
                                            check_pos = await exchange.fetch_positions([symbol])
                                            if len(check_pos) == 0 :
                                                df_open_trade_info.loc[symbol, 'status'] = "SL WARNING CLOSING"
                                            else : 
                                                pprint.pprint(f'WARNING CLOSE {symbol} MANUALY !!!')

                                                df_open_trade_info.loc[symbol, 'sl_id'] = df_open_trade_info.loc[symbol, 'new_sl_id']
                                                df_open_trade_info.loc[symbol, 'status'] = "SL WARNING "

                                                # exit()

                                        except Exception as e:

                                            alert = {'alert': f'trade {symbol} WARNING CLOSING {e}'}
                                            alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} sl3'])
                                            alert_df = pd.concat([alert_df, alert_data])
                                            alertnbr += 1

                                            df_open_trade_info.loc[symbol, 'sl_id'] = df_open_trade_info.loc[symbol, 'new_sl_id']
                                            df_open_trade_info.loc[symbol, 'status'] = "SL WARNING "
                                            # exit()
                        else : 

                            df_open_trade_info.loc[symbol, "sl_id"] = new_sl["id"]

                            if df_open_trade_info.loc[symbol, 'status'] == "set_new_sl" :

                                df_open_trade_info.loc[symbol, 'status'] = "warning_new_sl"
                    else : 
                                
                        df_open_trade_info.loc[symbol, 'status'] = "warning_new_sl"


                except Exception as e:
                                     
                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} sl4'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1

                    respord = None
                    params = {'stop' : True, "planType" : "normal_plan"}

                    respord.extend(await exchange.fetch_open_orders(symbol, params= params))
                    respord.extend(await exchange.fetch_open_orders(symbol))

                    if any('info' in resp and 'side' in resp['info'] and ("open_long" in resp['info']['side'] or "open_short" in resp['info']['side']) for resp in respord) :

                            try :
                                
                                params = {'stop' : True, "planType" : "normal_plan"}

                                await exchange.cancel_order(df_open_trade_info["id"].loc[symbol], symbol, params= params)

                                await asyncio.sleep(0.005)

                                df_open_trade_info.loc[symbol, 'status'] = "SL WARNING CLOSING"

                            except Exception as e :
                            
                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} sl5'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                                display(f'trade SL {symbol} NOT IN EXCHANGE 2')

                                try :
                            
                                    await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol)

                                    await asyncio.sleep(0.005)

                                    df_open_trade_info.loc[symbol, 'status'] = "SL WARNING CLOSING"

                                except Exception as e: 
                            
                                    alert = {'alert': f'trade {symbol} WARNING CLOSING 2 {e}'}
                                    alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} sl6'])
                                    alert_df = pd.concat([alert_df, alert_data])
                                    alertnbr += 1
                                    exit()

In [11]:
# OUVERTURE DES ORDRES

async def trade_opener() :

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop
    global alertnbr
    global alert_df

    if (top_10_values is not None and len(fulldata) > wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind 
        and not data8.empty and data8.iloc[-1].notnull().any() and free_balance > 50 and nbr_open_trade <= nbr_max_trade) :

        # for symbol, ticker in tickers.items() :
        for symbol in top_10_values.keys():
       
            trade = []

            # pprint.pprint(f'tentative pour {symbol}')
            
            if top_10_values[symbol][1] not in df_open_trade_info.index and data8[symbol + ' Tradable'].iloc[-1] == 'open_long' :

                # pprint.pprint(f'tentative data8 pour {symbol}')

                try :

                    trade = [
                        {
                            'symbol': top_10_values[symbol][1],
                            "type": 'limit',
                            'side': 'open_long',
                            'amount': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                            "price": exchange.price_to_precision(symbol, fulldata[symbol + ' ask'].iloc[-1] + (swap_markets[symbol]['precision']['price'] *20)),
                            'params' : {
                                "marginCoin": "USDT",
                                'side': 'open_long',
                                'force': "post_only",
                                "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' last'].iloc[-1] + (swap_markets[symbol]['precision']['price']*10)),
                                'triggerType': 'market_price',
                                "leverage": "1",
                                "planType" : "normal_plan",
                                'clientOid': f"Trade_nb_{trade_nb}"
                            }
                        },
                        {
                            'symbol': top_10_values[symbol][1],
                            "type": 'limit',
                            'side': 'close_long',
                            'amount': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                            "price": exchange.price_to_precision(symbol, fulldata[symbol + ' bid'].iloc[-1] * (1 - trail_stop) - (swap_markets[symbol]['precision']['price'] * 20)),
                            'params' :{
                                "marginCoin": "USDT",
                                'side': 'close_long',
                                "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' last'].iloc[-1] * (1 - trail_stop) - (swap_markets[symbol]['precision']['price']*10)),
                                "triggerType":"market_price",
                                "reduceOnly" : True,
                                "leverage": "1",
                                "planType" : "normal_plan",
                                'clientOid': f"Trade_nb_{trade_nb}_slV_1"
                            }
                        }]
                
                except Exception as e:
                                     
                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1 {symbol}'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1
                    continue

                
                id = None
                sl_id = None
                response_create_sl = None
                response_create_ord = None

                try :
                    
                    response_create_sl = await exchange.create_order(trade[1]['symbol'], trade[1]["type"], trade[1]['side'], trade[1]['amount'], trade[1]['price'], trade[1]['params'])
                
                    await asyncio.sleep(0.01)

                    if response_create_sl['id'] :
                        
                        sl_id = response_create_sl['id']
                        
                        await asyncio.sleep(0.005)

                        response_open_ord = []

                        params = {"stop" : True}
                        response_open_ord.extend(await exchange.fetch_open_orders(symbol, params= params))
                        await asyncio.sleep(0.005)
                        response_open_ord.extend(await exchange.fetch_open_orders(symbol))
                        await asyncio.sleep(0.01)

                        if len(response_open_ord) == 1 and response_open_ord[0]['id'] == sl_id : 

                            try :
                    
                                response_create_ord= await exchange.create_order(trade[0]['symbol'], trade[0]["type"], trade[0]['side'], trade[0]['amount'], trade[0]['price'], trade[0]['params'])
                
                                await asyncio.sleep(0.01)

                                if response_create_ord['id'] :
                        
                                    id = response_create_ord['id']
                        
                                    await asyncio.sleep(0.005)

                                    df_trade_row = pd.DataFrame({
                                        'symbol': [trade[0]['symbol']],
                                        'amount' : [float(trade[0]['amount'])],
                                        'side': [trade[0]['side']],
                                        'sl_side': [trade[1]['side']],
                                        'opening_price': [float(trade[0]['price'])],
                                        'sl_triggerPrice': [float(trade[1]['params']['triggerPrice'])],
                                        'sl_Execut_Price': [float(trade[1]['price'])],
                                        'clientOid' : [trade[0]['params']['clientOid']],
                                        'sl_clientOid' : [trade[1]['params']['clientOid']],
                                        'id': [id],
                                        "sl_id" : [sl_id],
                                        "new_sl_id" : "",
                                        'status' : "starting",
                                        'sl_version' : [1],
                                        "sym" : symbol
                                        })
                    
                                    # pprint.pprint(df_trade_row)
                                    df_trade_row.set_index('symbol', inplace=True)
                                    # pprint.pprint(f'index is ok')
                                    # df_trade_row['sl_triggerPrice'] = df_trade_row['sl_triggerPrice'].apply(pd.to_numeric)
                                    df_open_trade_info = pd.concat([df_open_trade_info, df_trade_row])
                                    # pprint.pprint(df_open_trade_info)
                                    nbr_open_trade += 1
                                    trade_nb += 1 

                                    await refresh_balance()
                                
                                else :

                                    try :

                                        params = {"planType" : "normal_plan"}
                                        await exchange.cancel_order(sl_id, symbol, params= params)
                                        await exchange.cancel_order(sl_id, symbol)

                                    except Exception as e:

                                        alert = {'alert': str(e)}
                                        alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1.2 {symbol}'])
                                        alert_df = pd.concat([alert_df, alert_data])
                                        alertnbr += 1

                            except Exception as e:
                                     
                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1 {symbol}'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                                try :

                                    params = {"planType" : "normal_plan"}
                                    await exchange.cancel_order(sl_id, symbol, params= params)
                                    await exchange.cancel_order(sl_id, symbol)

                                except Exception as e:

                                    alert = {'alert': str(e)}
                                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1.2 {symbol}'])
                                    alert_df = pd.concat([alert_df, alert_data])
                                    alertnbr += 1

                        else :

                            try :

                                params = {"planType" : "normal_plan"}
                                await exchange.cancel_order(sl_id, symbol, params= params)
                                await exchange.cancel_order(sl_id, symbol)

                            except Exception as e:

                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1.3 {symbol}'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                except Exception as e:

                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open2 {symbol}'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1

            elif top_10_values[symbol][1] not in df_open_trade_info.index and data8[symbol + ' Tradable'].iloc[-1] == 'open_short':

                # pprint.pprint(f'tentative data8 pour {symbol}')
            
                trade = [
                    {
                        'symbol': top_10_values[symbol][1],
                        "type": 'limit',
                        'side': 'open_short',
                        'amount': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        "price": exchange.price_to_precision(symbol, fulldata[symbol + ' bid'].iloc[-1] - (swap_markets[symbol]['precision']['price'] * 20)),
                        'params' : {
                            "marginCoin": "USDT",
                            'side': 'open_short',
                            'force': "post_only",
                            "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' last'].iloc[-1] - (swap_markets[symbol]['precision']['price']*10)),
                            "triggerType":"market_price",
                            "leverage": "1",
                            "planType" : "normal_plan",
                            'clientOid': f"Trade_nb_{trade_nb}"
                        }
                    },
                    {
                        'symbol': top_10_values[symbol][1],
                        "type": 'limit',
                        'side': 'close_short',
                        'amount': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        "price": exchange.price_to_precision(symbol, fulldata[symbol + ' ask'].iloc[-1] * (1 + trail_stop) + (swap_markets[symbol]['precision']['price'] * 20)),
                        'params' : {
                            "marginCoin": "USDT",
                            'side': 'close_short',
                            "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' last'].iloc[-1] * (1 + trail_stop) + (swap_markets[symbol]['precision']['price']*10)),
                            "triggerType":"market_price",
                            "reduceOnly" : True,
                            "leverage": "1",
                            "planType" : "normal_plan",
                            'clientOid': f"Trade_nb_{trade_nb}_slV_1"
                        }
                    }]

                id = None
                sl_id = None
                response_create_sl = None
                response_create_ord = None

                try :
                    
                    response_create_sl = await exchange.create_order(trade[1]['symbol'], trade[1]["type"], trade[1]['side'], trade[1]['amount'], trade[1]['price'], trade[1]['params'])
                
                    await asyncio.sleep(0.01)

                    if response_create_sl['id'] :
                        
                        sl_id = response_create_sl['id']
                        
                        await asyncio.sleep(0.005)

                        response_open_ord = []

                        params = {"stop" : True}
                        response_open_ord.extend(await exchange.fetch_open_orders(symbol, params= params))
                        await asyncio.sleep(0.005)
                        response_open_ord.extend(await exchange.fetch_open_orders(symbol))
                        await asyncio.sleep(0.01)

                        if len(response_open_ord) == 1 and response_open_ord[0]['id'] == sl_id : 

                            try :
                    
                                response_create_ord= await exchange.create_order(trade[0]['symbol'], trade[0]["type"], trade[0]['side'], trade[0]['amount'], trade[0]['price'], trade[0]['params'])
                
                                await asyncio.sleep(0.01)

                                if response_create_ord['id'] :
                        
                                    id = response_create_ord['id']
                        
                                    await asyncio.sleep(0.005)

                                    df_trade_row = pd.DataFrame({
                                        'symbol': [trade[0]['symbol']],
                                        'amount' : [float(trade[0]['amount'])],
                                        'side': [trade[0]['side']],
                                        'sl_side': [trade[1]['side']],
                                        'opening_price': [float(trade[0]['price'])],
                                        'sl_triggerPrice': [float(trade[1]['params']['triggerPrice'])],
                                        'sl_Execut_Price': [float(trade[1]['price'])],
                                        'clientOid' : [trade[0]['params']['clientOid']],
                                        'sl_clientOid' : [trade[1]['params']['clientOid']],
                                        'id': [id],
                                        "sl_id" : [sl_id],
                                        "new_sl_id" : "",
                                        'status' : "starting",
                                        'sl_version' : [1],
                                        "sym" : symbol
                                        })
                    
                                    # pprint.pprint(df_trade_row)
                                    df_trade_row.set_index('symbol', inplace=True)
                                    # pprint.pprint(f'index is ok')
                                    # df_trade_row['sl_triggerPrice'] = df_trade_row['sl_triggerPrice'].apply(pd.to_numeric)
                                    df_open_trade_info = pd.concat([df_open_trade_info, df_trade_row])
                                    # pprint.pprint(df_open_trade_info)
                                    nbr_open_trade += 1
                                    trade_nb += 1 

                                    await refresh_balance()
                                
                                else :

                                    try :

                                        params = {"planType" : "normal_plan"}
                                        await exchange.cancel_order(sl_id, symbol, params= params)
                                        await exchange.cancel_order(sl_id, symbol)

                                    except Exception as e:

                                        alert = {'alert': str(e)}
                                        alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1.2 {symbol}'])
                                        alert_df = pd.concat([alert_df, alert_data])
                                        alertnbr += 1

                            except Exception as e:
                                     
                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1 {symbol}'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                                try :

                                    params = {"planType" : "normal_plan"}
                                    await exchange.cancel_order(sl_id, symbol, params= params)
                                    await exchange.cancel_order(sl_id, symbol)

                                except Exception as e:

                                    alert = {'alert': str(e)}
                                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1.2 {symbol}'])
                                    alert_df = pd.concat([alert_df, alert_data])
                                    alertnbr += 1

                        else :

                            try :

                                params = {"planType" : "normal_plan"}
                                await exchange.cancel_order(sl_id, symbol, params= params)
                                await exchange.cancel_order(sl_id, symbol)

                            except Exception as e:

                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1.3 {symbol}'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                except Exception as e:

                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open2 {symbol}'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1

In [12]:
# PARAMETRAGE

nbr_max_trade = 15
time_to_fill = 45
min_occur = 3
div_min_top = 100000
trail_stop = 0.001

wind_Var_Price = 3
wind_EMA1 = 4
wind_Var_EMA1 = 3
wind_EMA2 = 9
wind_Var_EMA2 = 4
wind_Div_Price_EMA1 = 4
wind_Div_EMA1_2 = 6
wind_Div_Moy_Price_EMA1 = 2
wind_Div_Moy_EMA1_2 = 2

occur_wind = 20

trade_nb = 0
nbr_open_trade = 0
alertnbr = 1

# INITIALISATION DES DATAFRAMES

data1 = pd.DataFrame()
data2 = pd.DataFrame()
data3 = pd.DataFrame()
data4 = pd.DataFrame()
data5 = pd.DataFrame()
data6 = pd.DataFrame()
data7 = pd.DataFrame()
data8 = pd.DataFrame()
last_20 = pd.DataFrame()
alert_df = pd.DataFrame()
fulldata = pd.DataFrame()
df_open_trade_info = pd.DataFrame()
df_closed_trade_info = pd.DataFrame()

In [13]:
async def main():

    global wind_Var_Price 
    global wind_EMA1 
    global wind_Var_EMA1
    global wind_EMA2
    global wind_Var_EMA2
    global wind_Div_Price_EMA1
    global wind_Div_EMA1_2
    global wind_Div_Moy_Price_EMA1
    global wind_Div_Moy_EMA1_2

    global occur_wind
    global min_occur
    global div_min_top
    global trail_stop
    global nbr_max_trade
    global nbr_open_trade
    global trade_nb
    global alert_df
    global alertnbr
    global time_to_fill

    global last_20

    global data1
    global data2
    global data3 
    global data4 
    global data5
    global data6
    global data7
    global data8
    global fulldata
    global df_open_trade_info
    global df_closed_trade_info

    while True:

        try:

            await refresh_balance()
            display(last_20.tail(3))
            display(df_closed_trade_info)
            display(df_open_trade_info)
            display(f'Balance is {balance}')
            display(alert_df.tail(5))
            await config_strikerV2()
            await asyncio.sleep(0.005)
            await cleaner()
            await asyncio.sleep(0.005)
            await live_prices()
            await asyncio.sleep(0.005)
            await indicators_calculation()
            await asyncio.sleep(0.005)
            await time_striker()
            await asyncio.sleep(0.005)
            await cleaner()
            await asyncio.sleep(0.005)
            await sl_loader()
            await asyncio.sleep(0.005)
            await cleaner()
            await asyncio.sleep(0.005)
            await trade_opener()
            await asyncio.sleep(0.005)

        except KeyboardInterrupt:

            display(f'Interuption du programme ')
            await exchange.close()
            break

        clear_output(wait=True)

        await asyncio.sleep(0.001)

await main()

,BTC/USDT:USDT last,BTC/USDT:USDT ask,BTC/USDT:USDT bid,ETH/USDT:USDT last,ETH/USDT:USDT ask,ETH/USDT:USDT bid,XRP/USDT:USDT last,XRP/USDT:USDT ask,XRP/USDT:USDT bid,EOS/USDT:USDT last,...,PYTH/USDT:USDT bid,JTO/USDT:USDT last,JTO/USDT:USDT ask,JTO/USDT:USDT bid,1000SATS/USDT:USDT last,1000SATS/USDT:USDT ask,1000SATS/USDT:USDT bid,1000RATS/USDT:USDT last,1000RATS/USDT:USDT ask,1000RATS/USDT:USDT bid
2023-12-16 21:12:07,42380.8,42380.8,42380.7,2240.13,2240.13,2239.88,0.62098,0.62098,0.62081,0.8017,...,0.3892,2.9613,2.9628,2.9599,0.000756,0.000756,0.000755,0.3949,0.39467,0.39386
2023-12-16 21:12:09,42380.8,42380.8,42380.7,2240.13,2240.13,2239.96,0.62098,0.62098,0.62081,0.8017,...,0.3892,2.9613,2.9627,2.9604,0.000755,0.000756,0.000755,0.3949,0.39451,0.39365
2023-12-16 21:12:11,42380.7,42380.8,42380.7,2239.92,2239.89,2239.88,0.62098,0.62098,0.62081,0.8017,...,0.3890,2.9613,2.9628,2.9608,0.000755,0.000756,0.000755,0.3949,0.39633,0.39542


""


""


'Balance is 190.32980252'

""


CancelledError: 

In [14]:
params = {"stop" : True}
await exchange.fetch_positions(['DASHUSDT_UMCBL'])

[]

In [17]:
response_pos = [[]]
pprint.pprint(len(await exchange.fetch_positions(['DASHUSDT_UMCBL'])))

0


In [13]:
response = []

params = {"stop" : True}
response.extend(await exchange.fetch_open_orders('DASHUSDT_UMCBL', params=params))
await asyncio.sleep(0.005)
response.extend(await exchange.fetch_open_orders('DASHUSDT_UMCBL'))
await asyncio.sleep(0.01)

pprint.pprint(len(response))


0
